In [1]:
# Mục đích: Cài đặt các thư viện cần thiết cho mô hình học sâu và xử lý dữ liệu.
# transformers: Thư viện của Hugging Face dùng để làm việc với các mô hình học sâu như BERT, GPT, v.v.
# datasets: Thư viện để tải và xử lý dữ liệu huấn luyện một cách hiệu quả.
# torch, torchvision: Thư viện PyTorch dùng để xây dựng và huấn luyện mô hình học sâu, trong đó torchvision hỗ trợ xử lý dữ liệu hình ảnh.
!pip install transformers datasets torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Import hàm load_dataset từ thư viện datasets để tải các bộ dữ liệu có sẵn hoặc được chỉ định.
from datasets import load_dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Tải dữ liệu từ thư mục Google Drive
# Chú ý: Thư mục `processed_sports_data` cần chứa dữ liệu được tổ chức theo cấu trúc phù hợp với ImageFolder.
datasets = load_dataset(
    "imagefolder",
    data_dir="./drive/MyDrive/processed_sports_data"
)

# Kiểm tra thông tin dữ liệu đã tải
print(datasets)

Resolving data files:   0%|          | 0/8319 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 8319
    })
})


In [5]:
# labels: Danh sách các nhãn (labels) từ dữ liệu huấn luyện.
# label2id: Từ điển ánh xạ nhãn sang ID (dạng chuỗi).
# id2label: Từ điển ánh xạ ID sang nhãn.
labels = datasets["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
# Kiểm tra ánh xạ
print("Label to ID:", label2id)
print("ID to Label:", id2label)

Label to ID: {'badminton': '0', 'cricket': '1', 'football': '2', 'karate': '3', 'swimming': '4', 'tennis': '5', 'wrestling': '6'}
ID to Label: {'0': 'badminton', '1': 'cricket', '2': 'football', '3': 'karate', '4': 'swimming', '5': 'tennis', '6': 'wrestling'}


In [6]:
# shuffle(seed=1012): Trộn dữ liệu với một seed cố định để kết quả trộn có thể tái lập.
shuffled_dataset = datasets.shuffle(seed=2003)

In [7]:
# Xem 10 nhãn đầu tiên trong tập dữ liệu sau khi shuffle
print("10 nhãn đầu tiên sau khi shuffle:")
print(shuffled_dataset['train']['label'][:10])

10 nhãn đầu tiên sau khi shuffle:
[5, 6, 2, 2, 1, 1, 4, 5, 2, 3]


In [8]:
# Chia tập dữ liệu train thành train và test với tỷ lệ 80:20
train_test = shuffled_dataset['train'].train_test_split(test_size=0.2)

# Kiểm tra kích thước của từng tập
print(f"Số lượng ảnh trong tập train: {len(train_test['train'])}")
print(f"Số lượng ảnh trong tập test: {len(train_test['test'])}")

Số lượng ảnh trong tập train: 6655
Số lượng ảnh trong tập test: 1664


In [9]:
from huggingface_hub import login

# Đăng nhập vào Hugging Face (đảm bảo bạn đã có tài khoản và token)
login()

In [10]:
# Push tập dữ liệu train_test lên Hugging Face Hub
train_test.push_to_hub("vieanh/sports_img_classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/3328 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

Map:   0%|          | 0/3327 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1664 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/vieanh/sports_img_classification/commit/02d73bf16c9ff9e98847df2507ada4ba0e382211', commit_message='Upload dataset', commit_description='', oid='02d73bf16c9ff9e98847df2507ada4ba0e382211', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vieanh/sports_img_classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vieanh/sports_img_classification'), pr_revision=None, pr_num=None)